In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import re
import requests
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
client = MongoClient("localhost:27017")

In [3]:
db = client["Ironhack"]

In [4]:
comp = db.get_collection("companies")

## Getting the address of each company in palo alto

In [5]:
def palo_alto_full_address ():
    companies_name   = {'name':
                            {"$regex": 
                            '(SurveyMonkey|LivingSocial|Zynga|BrightSource|Better Place|Tesla Motors|SolarCity|Telefonica|Comcast|SunEdison)',
                            "$options" :'i'}}
    companies_city   = {'offices.city':'Palo Alto'}
    companies_city_1 = {'offices.city':'San Francisco'}                

    project_= {'offices.address1':1,'offices.city':1,'_id':0}
    project_1= {'name':1,'offices.address1':1,'offices.city':1,'_id':0}
    address_with_name = list(comp.find({ "$and": [companies_name,{'$or':[companies_city,companies_city_1]}]},project_1))

    return address_with_name

In [6]:
palo_alto_full_address ()

[{'name': 'Tesla Motors',
  'offices': [{'address1': '3500 Deer Creek rd.', 'city': 'Palo Alto'}]},
 {'name': 'Zynga',
  'offices': [{'address1': '365 Vermont St.', 'city': 'San Francisco'},
   {'address1': '', 'city': ''},
   {'address1': '', 'city': 'Baltimore'},
   {'address1': '', 'city': 'Los Angeles'}]},
 {'name': 'Better Place',
  'offices': [{'address1': '1070 Arastradero Road,', 'city': 'Palo Alto'}]},
 {'name': 'SurveyMonkey',
  'offices': [{'address1': '111 SW 5th Ave', 'city': 'Portland'},
   {'address1': '285 Hamilton Avenue', 'city': 'Palo Alto'},
   {'address1': 'Rua Ivens. 3, Edificio D.', 'city': 'Funchal'}]}]

In [7]:
def palo_alto_address ():
    companies_name   = {'name':
                            {"$regex": 
                            '(SurveyMonkey|LivingSocial|Zynga|BrightSource|Better Place|Tesla Motors|SolarCity|Telefonica|Comcast|SunEdison)',
                            "$options" :'i'}}
    companies_city   = {'offices.city':'Palo Alto'}
    companies_city_1 = {'offices.city':'San Francisco'}                

    project_= {'offices.address1':1,'offices.city':1,'_id':0}
    project_1= {'name':1,'offices.address1':1,'offices.city':1,'_id':0}
    address = list(comp.find({ "$and": 
                [companies_name,{'$or':
                                 [companies_city,companies_city_1]}]},project_ ))
    return address


In [33]:
address= palo_alto_address ()
address

[{'offices': [{'address1': '3500 Deer Creek rd.', 'city': 'Palo Alto'}]},
 {'offices': [{'address1': '365 Vermont St.', 'city': 'San Francisco'},
   {'address1': '', 'city': ''},
   {'address1': '', 'city': 'Baltimore'},
   {'address1': '', 'city': 'Los Angeles'}]},
 {'offices': [{'address1': '1070 Arastradero Road,', 'city': 'Palo Alto'}]},
 {'offices': [{'address1': '111 SW 5th Ave', 'city': 'Portland'},
   {'address1': '285 Hamilton Avenue', 'city': 'Palo Alto'},
   {'address1': 'Rua Ivens. 3, Edificio D.', 'city': 'Funchal'}]}]

In [9]:
def d_address(address):
    address_lst= []
    for i in address:
        for key, value in i.items():
            for j in value:
                address_lst.append(j)
    df_address= pd.DataFrame(address_lst)
    df_address=df_address.apply(lambda x: x[df_address['city'].isin(['Palo Alto', 'San Francisco'])])
   
    return df_address

In [10]:
d_address (address)

,address1,city
0,3500 Deer Creek rd.,Palo Alto
1,365 Vermont St.,San Francisco
5,"1070 Arastradero Road,",Palo Alto
7,285 Hamilton Avenue,Palo Alto


## Coordinates

In [11]:
where= '3500 Deer Creek rd., Palo Alto, United States of America'

In [12]:
url_geocode = f"https://geocode.xyz/{where}?json=1"
requests.get(url_geocode).json()

{'success': False,
 'error': {'code': '006',
  'message': 'Request Throttled. Over Rate limit: up to 2 per sec. See geocode.xyz/pricing',
  'requests': '64'}}

In [13]:
def lst_coordinates ():
    list_1= [
        {'name':'Tesla','lat':37.3942781,'lon':-122.152006817},
        {'name':'Better Place','lat':37.3942641 ,'lon':-122.14902917 },
        {'name':'SurveyMonkey','lat':37.4451016 ,'lon': -122.162978817},
        {'name':'Zynga', 'lat':37.76476,'lon':-122.406779318}
    ]
    df_coordinates= pd.DataFrame(list_1)
    return df_coordinates

In [14]:
df_coordinates = lst_coordinates ()

In [15]:
df_coordinates

,name,lat,lon
0,Tesla,37.394278,-122.152007
1,Better Place,37.394264,-122.149029
2,SurveyMonkey,37.445102,-122.162979
3,Zynga,37.764760,-122.406779


In [16]:
def map_palo_alto():
    palal_lat = 37.4256293
    palal_lon = -122.2053912

    palal_map = Map(location=[palal_lat, palal_lon], zoom_start = 9)
    return palal_map

In [17]:
palal_map = map_palo_alto()
palal_map

In [28]:
for index, row in df_coordinates.iterrows():
    ubications = {"location": [row["lat"], row["lon"]]}
    if row["name"] == "Tesla":        
        icon = Icon (
                    color="blue",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="briefcase",
                    icon_color = "black"
                    )
    elif row["name"] == "Better Place":
        icon = Icon (
                    color="red",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="briefcase",
                    icon_color = "yellow"
                    )
    elif row["name"] == "SurveyMonkey":
        icon = Icon (
                    color="purple",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="briefcase",
                    icon_color = "white"
                )
    elif row["name"] == "Zynga":
        icon = Icon (
                    color="white",
                    opacity = 0.6,
                    prefix = "fa",
                    icon="briefcase",
                    icon_color = "black"
                )

            #3. Marker
    new_marker = Marker(**ubications, icon = icon, radius = 200)

            #4. Add the Marker
    new_marker.add_to(palal_map)


In [29]:
palal_map

In [30]:
def ubication (df_coordinates):
    for index, row in df_coordinates.iterrows():
        ubications = {"location": [row["lat"], row["lon"]]}
        if row["name"] == "Tesla":        
            icon = Icon (
                        color="blue",
                        opacity = 0.6,
                        prefix = "fa",
                        icon="briefcase",
                        icon_color = "black"
                        )
        elif row["name"] == "Better Place":
            icon = Icon (
                        color="red",
                        opacity = 0.6,
                        prefix = "fa",
                        icon="briefcase",
                        icon_color = "yellow"
                        )
        elif row["name"] == "SurveyMonkey":
            icon = Icon (
                        color="purple",
                        opacity = 0.6,
                        prefix = "fa",
                        icon="briefcase",
                        icon_color = "white"
                    )
        elif row["name"] == "Zynga":
            icon = Icon (
                        color="white",
                        opacity = 0.6,
                        prefix = "fa",
                        icon="briefcase",
                        icon_color = "red"
                    )

                #3. Marker
        new_marker = Marker(**ubications, icon = icon, radius = 200)

                #4. Add the Marker
        new_marker.add_to(palal_map)
        return palal_map

In [31]:
ubication (df_coordinates)